<a href="https://colab.research.google.com/github/AriadnaVazquez/Metodos-Numericos-I/blob/main/Pivoteo%20parcial%20escalado.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np

def gaussian_elimination_with_significant_rounding(A, b, significant_digits=3):
    """
    Eliminación Gaussiana con pivoteo parcial escalado, aplicando redondeo
    a cifras significativas después de cada operación.

    Parámetros:
        A (numpy.ndarray): Matriz de coeficientes del sistema lineal.
        b (numpy.ndarray): Vector de términos independientes.
        significant_digits (int): Número de cifras significativas para redondeo.

    Retorna:
        numpy.ndarray: Solución del sistema lineal Ax = b.
    """
    def round_to_significant(x, sig_digits):
        """Redondeo a cifras significativas."""
        return np.round(x, sig_digits - int(np.floor(np.log10(abs(x)))) - 1) if x != 0 else 0

    n = len(b)
    A = A.copy()
    b = b.copy()

    # Redondear la matriz inicial y el vector independiente a cifras significativas
    A = np.vectorize(lambda x: round_to_significant(x, significant_digits))(A)
    b = np.vectorize(lambda x: round_to_significant(x, significant_digits))(b)

    # Extender la matriz A con b para realizar las operaciones simultáneamente
    Ab = np.hstack((A, b.reshape(-1, 1)))

    # Paso 1: Escalamiento y determinación inicial
    s = np.max(np.abs(A), axis=1)  # Vector de escalas
    if np.any(s == 0):
        raise ValueError("No existe solución única: escala cero detectada.")

    # Vector de índices que representa el orden de las filas
    NROW = np.arange(n)

    # Proceso de eliminación
    for i in range(n - 1):
        # Paso 2: Determinación del pivote escalado
        ratios = np.abs(Ab[NROW[i:], i]) / s[NROW[i:]]
        p = i + np.argmax(ratios)  # Índice del máximo elemento escalado

        # Intercambio de filas
        if p != i:
            NROW[i], NROW[p] = NROW[p], NROW[i]

        # Paso 3: Eliminación hacia adelante
        for j in range(i + 1, n):
            factor = round_to_significant(Ab[NROW[j], i] / Ab[NROW[i], i], significant_digits)
            Ab[NROW[j], i:] = np.vectorize(lambda x: round_to_significant(x, significant_digits))(
                Ab[NROW[j], i:] - factor * Ab[NROW[i], i:]
            )

    # Sustitución hacia atrás
    x = np.zeros(n)
    for i in range(n - 1, -1, -1):
        x[i] = round_to_significant(
            (Ab[NROW[i], -1] - np.dot(Ab[NROW[i], i + 1:n], x[i + 1:])) / Ab[NROW[i], i],
            significant_digits
        )

    return x


# Datos del sistema (matriz de coeficientes y vector independiente)
A = np.array([
    [2.11, -4.21, 0.921],
    [4.01, 10.2, -1.12],
    [1.09, 0.987, 0.832]
])

b = np.array([2.01, -3.09, 4.21])

# Resolviendo el sistema ajustando a 3 cifras significativas
solution_with_significant_rounding = gaussian_elimination_with_significant_rounding(A, b, significant_digits=3)

# Mostrando la solución
print("Solución del sistema:")
for i, val in enumerate(solution_with_significant_rounding, start=1):
    print(f"x_{i} = {val:.3f}")

Solución del sistema:
x_1 = -0.432
x_2 = 0.427
x_3 = 5.120
